# 속성 추출
1. 차트 진입후 5위 이내까지 진입한 일수
2. 해당 곡 아티스트의 다른 곡이 차트에 진입한 일수
3. 차트 진입 후 5위 이내까지 일간 랭킹 상승 평균치
4. and more...

In [1]:
from getAttribute import dbQuery, chartInDays, rankIncreaseMean, rankInDays

# 신경망 코드

In [2]:
import tensorflow as tf
import numpy as np

## Train dataset 추출
- 그날처럼(장덕철)
- WayBackHome(숀), 지나오다(닐로)
- WayBackHome(숀), 지나오다(닐로) 
<br/>를 조작된 데이터로 설정

In [3]:
geniedf = dbQuery("SELECT * FROM zuzak.genie where YYMMDD = 180105 and ranking <= 15")
bugsdf = dbQuery("SELECT * FROM zuzak.bugs where YYMMDD = 180710 and ranking <= 15")
mnetdf = dbQuery("SELECT * FROM zuzak.mnet where YYMMDD = 180701 and ranking <= 15")

# 데이터프레임 합치기
df = geniedf.append(bugsdf)
df = df.append(mnetdf)
df.reset_index(inplace=True, drop=True)

### 레이블 설정

In [4]:
# 조작음원 = 1
# 여기선 지나오다, way back home 조작으로 가정
tmp = df[(df['title']=='WayBackHome') | (df['title']=='지나오다') | (df['title']=='그날처럼')]
labels = np.zeros([45, 1])
labels[tmp.index] = 1

### 속성 추출 및 정규화
- 속성은 genie 차트에서 추출함

In [6]:
# 이전에 차트에 든 일수
chartindays = np.fromiter(map(chartInDays, df['title'], df['artist'], ['genie']*len(df)), np.int32)

In [7]:
# 랭킹 상승 평균값
rankimean = np.fromiter(map(rankIncreaseMean, df['title'], ['genie']*len(df)), np.int32)
rankimean = np.array((rankimean+1) / (chartindays+1))

In [8]:
# top5 진입 일수
rankindays = np.fromiter(map(rankInDays, df['title'], ['genie']*len(df)), np.int32)
rankindays = np.array((rankindays + 1) / (chartindays+1))

In [9]:
# 데이터 정규화
chartindays = np.array((chartindays+0.01) / (chartindays+1))
chartindays

array([0.01      , 0.9988475 , 0.505     , 0.01      , 0.99938547,
       0.99876712, 0.01      , 0.01      , 0.99907649, 0.505     ,
       0.98862069, 0.505     , 0.99936498, 0.97893617, 0.99932099,
       0.99953236, 0.01      , 0.99927843, 0.99543779, 0.99877778,
       0.99927843, 0.01      , 0.99973243, 0.99707965, 0.99810707,
       0.99922535, 0.99545872, 0.99888136, 0.01      , 0.01      ,
       0.01      , 0.99927843, 0.99922535, 0.99810707, 0.98848837,
       0.99927843, 0.99973243, 0.99706231, 0.01      , 0.99933691,
       0.01      , 0.99440678, 0.99326531, 0.01      , 0.01      ])

### 속성 합치기

In [10]:
train_data = np.array([chartindays, rankimean, rankindays]).T
train_data

array([[1.00000000e-02, 1.00000000e+00, 3.50000000e+01],
       [9.98847497e-01, 1.11757858e-01, 2.32828871e-03],
       [5.05000000e-01, 1.50000000e+00, 2.25000000e+01],
       [1.00000000e-02, 3.00000000e+00, 3.30000000e+01],
       [9.99385475e-01, 6.20732464e-02, 1.24146493e-03],
       [9.98767123e-01, 1.74346202e-02, 3.73599004e-03],
       [1.00000000e-02, 1.00000000e+00, 5.40000000e+01],
       [1.00000000e-02, 2.00000000e+00, 6.80000000e+01],
       [9.99076493e-01, 9.04850746e-02, 1.86567164e-03],
       [5.05000000e-01, 1.00000000e+00, 2.25000000e+01],
       [9.88620690e-01, 1.14942529e+00, 2.29885057e-02],
       [5.05000000e-01, 5.00000000e+01, 1.00000000e+00],
       [9.99364978e-01, 6.41436818e-02, 1.28287364e-03],
       [9.78936170e-01, 2.08510638e+00, 4.25531915e-02],
       [9.99320988e-01, 6.79012346e-02, 1.37174211e-03],
       [9.99532357e-01, 4.67642891e-02, 9.44733113e-04],
       [1.00000000e-02, 1.40000000e+01, 8.00000000e+00],
       [9.99278426e-01, 7.28862

# 신경망 학습 

In [181]:
tf.reset_default_graph()
# [chartindays, rankimean, rankindays]
x_data = train_data
y_data = labels

#########
# 신경망 모델 구성
######
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# W = tf.Variable(tf.random_uniform([3, 1], -1., 1.))
W1 = tf.Variable(tf.random_normal([3, 50]))
b1 = tf.Variable(tf.random_normal([50]))
L1 = tf.nn.sigmoid(tf.add(tf.matmul(X, W1), b1))

W2 = tf.Variable(tf.random_normal([50, 1]))
b2 = tf.Variable(tf.random_normal([1]))
L2 = tf.nn.sigmoid(tf.add(tf.matmul(L1, W2), b2))

model = L2

cost = tf.reduce_mean(tf.square(model-Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.02)
train_op = optimizer.minimize(cost)

# 신경망 모델 학습
                      
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())
                      
ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    for step in range(1000):
        sess.run(train_op, feed_dict={X: x_data, Y: y_data})
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))    

prediction = model
target = Y
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

INFO:tensorflow:Restoring parameters from ./model\dnn.ckpt
예측값: [[9.5636570e-01]
 [3.4701756e-08]
 [2.8884821e-03]
 [2.7722120e-02]
 [3.3975891e-08]
 [3.3721840e-08]
 [3.1576533e-02]
 [9.2012547e-03]
 [3.4368718e-08]
 [7.5380998e-03]
 [1.4632099e-07]
 [2.0843167e-04]
 [3.4002081e-08]
 [2.6838215e-06]
 [3.4051730e-08]
 [3.3793505e-08]
 [9.8995918e-01]
 [3.4116155e-08]
 [3.4674894e-08]
 [3.4868616e-08]
 [3.4099628e-08]
 [2.3128474e-03]
 [3.3582875e-08]
 [3.3908950e-08]
 [3.6097610e-08]
 [3.4179656e-08]
 [3.4225980e-08]
 [3.3560593e-08]
 [4.8698247e-03]
 [1.8607525e-04]
 [2.3128474e-03]
 [3.4116155e-08]
 [3.4179656e-08]
 [3.6097610e-08]
 [3.5627487e-08]
 [3.4099628e-08]
 [3.3582875e-08]
 [3.8769723e-08]
 [1.8494744e-04]
 [3.4038482e-08]
 [2.3128474e-03]
 [3.4433413e-08]
 [3.4660147e-08]
 [2.3128474e-03]
 [9.9557883e-01]]
실제값: [[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0

# 테스트 데이터
- bugs 차트에서의 WayBackHome, 지나오다에 대해 테스트
- 훈련 데이터는 genie차트에 대해 훈련함

In [172]:
df = dbQuery("SELECT * FROM zuzak.genie where ranking <=11 and YYMMDD = 180728")

### 데이터 정규화

In [175]:
# 이전에 차트에 든 일수
chartindays = np.fromiter(map(chartInDays, df['title'], df['artist'], ['mnet']*len(df)), np.int32)
# 랭킹 상승 평균값
rankimean = np.fromiter(map(rankIncreaseMean, df['title'], ['mnet']*len(df)), np.int32)
rankimean = np.array((rankimean+0.01) / (chartindays+1))
# top5 진입 일수
rankindays = np.fromiter(map(rankInDays, df['title'], ['mnet']*len(df)), np.int32)
rankindays = np.array((rankindays + 0.01) / (chartindays+1))
# 정규화
chartindays = np.array((chartindays+0.01) / (chartindays+1))
# 데이터 합치기
test_data = np.array([chartindays, rankimean, rankindays]).T

### 테스트

In [176]:
test_data

array([[1.00000000e-02, 1.00100000e+01, 1.00100000e+01],
       [9.99187192e-01, 8.12889984e-02, 8.29228243e-04],
       [9.99548564e-01, 4.46922025e-02, 4.60556316e-04],
       [9.99270987e-01, 7.21723122e-02, 7.43740795e-04],
       [9.95975610e-01, 1.22357724e-02, 1.63008130e-02],
       [9.99713542e-01, 2.86487269e-02, 2.92245370e-04],
       [9.97670588e-01, 2.28258824e-01, 2.37647059e-03],
       [9.99187192e-01, 7.96469622e-02, 8.29228243e-04],
       [9.01000000e-01, 6.01000000e-01, 9.01000000e-01],
       [9.98096154e-01, 1.15576923e-02, 3.86538462e-03],
       [1.00000000e-02, 5.01000000e+00, 1.50100000e+01]])

In [177]:
x_data = test_data

result = sess.run(prediction, feed_dict={X: x_data})
result = result.astype(np.float32)

print('예측값:', result)

예측값: [[9.8925650e-01]
 [3.4221603e-08]
 [3.3764771e-08]
 [3.4099500e-08]
 [3.4354887e-08]
 [3.3595626e-08]
 [3.7053084e-08]
 [3.4202028e-08]
 [3.6728471e-07]
 [3.3799438e-08]
 [9.1404313e-01]]


In [178]:
df['prediction'] = result
df

,YYMMDD,ranking,title,artist,prediction
0,180728,1,WayBackHome,숀(SHAUN),9.892565e-01
1,180728,2,뚜두뚜두(DDU-DUDDU-DU),BLACKPINK,3.422160e-08
2,180728,3,DanceTheNightAway,TWICE(트와이스),3.376477e-08
3,180728,4,너나해(Egotistic),마마무(Mamamoo),3.409950e-08
4,180728,5,1도없어,Apink(에이핑크),3.435489e-08
5,180728,6,여행,볼빨간사춘기,3.359563e-08
6,180728,7,LoveU,청하,3.705308e-08
7,180728,8,ForeverYoung,BLACKPINK,3.420203e-08
8,180728,9,"모든날,모든순간(Everyday,EveryMoment)",폴킴,3.672847e-07
9,180728,10,동화,멜로망스(MeloMance),3.379944e-08


## 모델 저장

In [179]:
saver = tf.train.Saver(tf.global_variables())
saver.save(sess, './model/dnn.ckpt')

'./model/dnn.ckpt'